In [73]:
import pandas as pd
import numpy as np

import os
import glob

import re

# pd.options.display.max_columns = None

## Load Data

In [2]:
mydir = r"D:\PURWADHIKA\FINAL PROJECT\ecommerce-analysis\data\raw"

file_list = glob.glob(mydir + "\*.csv")
file_list 

['D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_customers_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_geolocation_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_orders_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_order_items_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_order_payments_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_order_reviews_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_products_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\olist_sellers_dataset.csv',
 'D:\\PURWADHIKA\\FINAL PROJECT\\ecommerce-analysis\\data\\raw\\product_category_name_translation.csv']

In [3]:
def efn(text):
    file = text.split('\\')[-1].replace('.csv','')
    return file

In [4]:
dfs = {f'{efn(file)}_df': pd.read_csv(file) for file in file_list}

In [16]:
for n in dfs.keys():
    print(n)

olist_customers_dataset_df
olist_geolocation_dataset_df
olist_orders_dataset_df
olist_order_items_dataset_df
olist_order_payments_dataset_df
olist_order_reviews_dataset_df
olist_products_dataset_df
olist_sellers_dataset_df
product_category_name_translation_df


## Data Understanding

In [55]:
dataInfo = []

for n in dfs.keys():
    dataInfo.append({
        'dataset_name' : n,
        'shape' : dfs[n].shape,
        'columns' : list(dfs[n].columns),
        'duplicate' : dfs[n].duplicated().sum(),
        'null' : dfs[n].isna().sum().sum(),
        'null_columns' : dfs[n].columns[dfs[n].isna().any()].tolist()
    })

pd.DataFrame(dataInfo)


,dataset_name,shape,columns,duplicate,null,null_columns
0,olist_customers_dataset_df,"(99441, 5)","[customer_id, customer_unique_id, customer_zip_code_prefix, customer_city, customer_state]",0,0,[]
1,olist_geolocation_dataset_df,"(1000163, 5)","[geolocation_zip_code_prefix, geolocation_lat, geolocation_lng, geolocation_city, geolocation_state]",261831,0,[]
2,olist_orders_dataset_df,"(99441, 8)","[order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date]",0,4908,"[order_approved_at, order_delivered_carrier_date, order_delivered_customer_date]"
3,olist_order_items_dataset_df,"(112650, 7)","[order_id, order_item_id, product_id, seller_id, shipping_limit_date, price, freight_value]",0,0,[]
4,olist_order_payments_dataset_df,"(103886, 5)","[order_id, payment_sequential, payment_type, payment_installments, payment_value]",0,0,[]
5,olist_order_reviews_dataset_df,"(99224, 7)","[review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp]",0,145903,"[review_comment_title, review_comment_message]"
6,olist_products_dataset_df,"(32951, 9)","[product_id, product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm]",0,2448,"[product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm]"
7,olist_sellers_dataset_df,"(3095, 4)","[seller_id, seller_zip_code_prefix, seller_city, seller_state]",0,0,[]
8,product_category_name_translation_df,"(71, 2)","[product_category_name, product_category_name_english]",0,0,[]


**olist_customers_dataset_df**

**olist_geolocation_dataset_df**

**olist_orders_dataset_df**

In [79]:
dfOrder = dfs['olist_orders_dataset_df']

In [80]:
# data type

dfOrder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [92]:
# convert to datetime

dateCol = ['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']

for c in dateCol:
    dfOrder[c] = pd.to_datetime(dfOrder[c])

dfOrder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [137]:
# check initial info of data
pd.DataFrame({
    'column': dfOrder.columns.values,
    'type': dfOrder.dtypes.values,
    'count': dfOrder.count(),
    'n_unique': dfOrder.nunique().values,
    'min': dfOrder.min().values,
    'max': dfOrder.max().values,
    'mod_values': [dfOrder[col].value_counts().nlargest(1).index.to_list()[0] for col in dfOrder.columns],
    'n_mod': [dfOrder[col].value_counts().nlargest(1).to_list()[0] for col in dfOrder.columns],
    'n_mod': [dfOrder[col].value_counts().nlargest(1).to_list()[0]/dfOrder[col].count() for col in dfOrder.columns],
    'sample_unique': [dfOrder[col].sort_values().unique() for col in dfOrder.columns]
})

,column,type,count,n_unique,min,max,mod_values,n_mod,sample_unique
order_id,order_id,object,99441,99441,00010242fe8c5a6d1ba2dd792cb16214,fffe41c64501cc87c801fd61db3f6244,e481f51cbdc54678b7cc49136f2d6af7,0.000010,"[00010242fe8c5a6d1ba2dd792cb16214, 00018f77f2f0320c557190d7a144bdd3, 000229ec398224ef6ca0657da4fc703e, 00024acbcdf0a6daa1e931b038114c75, 00042b26cf59d7ce69dfabb4e55b4fd9, 00048cc3ae777c65dbb7d2a0634bc1ea, 00054e8431b9d7675808bcb819fb4a32, 000576fe39319847cbb9d288c5617fa6, 0005a1a1728c9d785b8e2b08b904576c, 0005f50442cb953dcd1d21e1fb923495, 00061f2a7bc09da83e415a52dc8a4af1, 00063b381e2406b52ad429470734ebd5, 0006ec9db01a64e59a68b2c340bf65a7, 0008288aa423d2a3f00fcb17cd7d8719, 0009792311464db532ff765bf7b182ae, 0009c9a17f916a706d71784483a5d643, 000aed2e25dbad2f9ddb70584c5a2ded, 000c3e6612759851cc3cbb4b83257986, 000e562887b1f2006d75e0be9558292e, 000e63d38ae8c00bbcb5a30573b99628, 000e906b789b55f64edcb1f84030f90d, 000f25f4d72195062c040b12dce9a18a, 001021efaa8636c29475e7734483457d, 0010b2e5201cc5f1ae7e9c6cc8f5bd00, 0010dedd556712d7bb69a19cb7bbd37a, 00119ff934e539cf26f92b9ef0cdfed8, 0011d82c4b53e22e84023405fb467e57, 00125cb692d04887809806618a2a145f, 00130c0eee84a3d909e75bc08c5c3ca1, 0013503b13da1eac686219390b7d641b, 00137e170939bba5a3134e2386413108, 001427c0ec99cf8af737bd88e92fd444, 00143d0f86d6fbd9f9b38ab440ac16f5, 0014ae671de39511f7575066200733b7, 0015ebb40fb17286bea51d4607c4733c, 00169e31ef4b29deaae414f9a5e95929, 0016dfedd97fc2950e388d2971d718c7, 0017afd5076e074a48f1f1a4c7bac9c5, 001862358bf858722e1e2ae000cfed8b, 0019c29108428acffd089c36103c9440, 001ab0a7578dd66cd4b0a71f5b6e1e41, 001ac194d4a326a6fa99b581e9a3d963, 001b76dd48a5b1eee3e87778daa40df8, 001c85b5f68d2be0cb0797afc9e8ce9a, 001d8f0e34a38c37f7dba2a37d4eba8b, 001daeb0eddc45b999bad0801ad9d273, 001dbc16dc51075e987543d23a0507c7, 001e7ba991be1b19605ca0316e7130f9, 001e7cf2ad6bef3ade12ebc56ceaf0f3, 0020262c8a370bd5a174ea6a2a267321, 0020a222f55eb79a372d0efee3cca688, 002175704e8b209f61b9ad5cfd92b60e, 00229e4e43f7a7e0b9dd819ad43268d3, 0025081dcf9330f9a5052ae82c6ce396, 00254baeb6c932b0a8aeead91fbd02b5, 00259a44fcad3fc0474329e925d14fc3, 0025c5d1a8ca53a240ec2634bb4492ea, 002611a77fe03d076285fd4ca95db77c, 002691433f09002ac9ca0c4..."
customer_id,customer_id,object,99441,99441,00012a2ce6f8dcda20d059ce98491703,ffffe8b65bbe3087b653a978c870db99,9ef432eb6251297304e76186b10a928d,0.000010,"[00012a2ce6f8dcda20d059ce98491703, 000161a058600d5901f007fab4c27140, 0001fd6190edaaf884bcaf3d49edf079, 0002414f95344307404f0ace7a26f1d5, 000379cdec625522490c315e70c7a9fb, 0004164d20a9e969af783496f3408652, 000419c5494106c306a97b5635748086, 00046a560d407e99b969756e0b10f282, 00050bf6e01e69d5c0fd612f1bcfb69c, 000598caf2ef4117407665ac33275130, 0005aefbb696d34b3424dccd0a0e9fd0, 00062b33cb9f6fe976afdcff967ea74d, 00066ccbe787a588c52bd5ff404590e3, 00072d033fe2e59061ae5c3aff1a2be5, 0009a69b72033b2d0ec8c69fc70ef768, 000bf8121c3412d3057d32371c5d3395, 000e943451fc2788ca6ac98a682f2f49, 000f17e290c26b28549908a04cfe36c1, 000fd45d6fedae68fc6676036610f879, 0010068a73b7c56da5758c3f9e5c7375, 001028b78fd413e19704b3867c369d3a, 00104a47c29da701ce41ee52077587d9, 001051abfcfdbed9f87b4266213a5df1, 00114026c1b7b52ab1773f317ef4880b, 001226b2341ef620415ce7bbafcfac28, 0012a5c13793cf51e253f096a7e740dd, 0013280441d86a4f7a8006efdaf1b0fe, 0013cd8e350a7cc76873441e431dd5ee, 001450ebb4a77efb3d68be5f7887cb1e, 00146ad30454993879adaa91c518f68c, 00155f0530cc7b2bf73cc3f81cb01c52, 001574cd5824c0b1ea90dd4f4ba6d5b8, 0015bc9fd2d5395446143e8b215d7c75, 0015f7887e2fde13ddaa7b8e385af919, 0017a0b4c1f1bdb9c395fa0ac517109c, 0018c09f333634ca9c80d9ff46e43e9c, 0018f7fde63ca77254dbc47f4ea358b2, 0019c9aaad15b043c48f0a1180f22ce8, 001a57041f56400917a187dd74e6cbc1, 001b4e9b73aa98a6276b241bfd496822, 001c7f05398c45b42eee0d5a77783bca, 001df1ee5c36767aa607001ab1a13a06, 001f150aebb5d897f2059b0460c38449, 001f35d9f262c558fd065346fbf5801d, 001f6f1a5e902ad14e1f709a7215de11, 00205ad9ba1ef4340cef86583294cf82, 002236c4f333bc8df6a5939749eb7869, 002348c1099e3229276c8ad7d4ddc702, 0

In [91]:
dfOrder.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


**olist_order_items_dataset_df**

**olist_order_payments_dataset_df**

**olist_order_reviews_dataset_df**

In [158]:
review = dfs['olist_order_reviews_dataset_df']

review[review.duplicated(subset='order_id', keep=False)]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
30,540e7bbb2d06cfb7f85f3a88ba7ac97f,cf73e2cb1f4a9480ed70c154da3d954a,5,NaN,NaN,2018-01-18 00:00:00,2018-01-18 19:12:30
344,a0a641414ff718ca079b3967ef5c2495,169d7e0fd71d624d306f132acd791cbe,5,NaN,NaN,2018-03-04 00:00:00,2018-03-06 20:12:53
498,505a882ba08a689682a4afc6eb4e5965,1c308eca3f339414a92e518e2a2e5ee9,2,NaN,NaN,2017-12-28 00:00:00,2017-12-31 20:25:02
764,c5976a5a98e854fb23d7e03c6754ae60,2002ea16e75277eaa0b5d78632048540,5,NaN,NaN,2017-08-08 00:00:00,2017-08-10 11:11:29
778,62c7722239b976d943ec0d430cfe890e,1d297b4800ed1a3c5b0944d84c01ee99,3,NaN,NaN,2017-10-22 00:00:00,2017-10-31 15:33:32
...,...,...,...,...,...,...,...
98989,dfb3db02188d809d5cd199496b6da87e,c0db7d31ace61fc360a3eaa34dd3457c,5,NaN,NaN,2018-02-17 00:00:00,2018-02-19 19:29:19
99108,2c6c08892b83ba4c1be33037c2842294,42ae1967f68c90bb325783ac55d761ce,4,NaN,"Chegou um pouco amassada, mas nada de mais, e chegou antes do prazo.",2017-07-03 00:00:00,2017-07-05 19:06:59
99164,2afe63a67dfd99b3038f568fb47ee761,c5334d330e36d2a810a7a13c72e135ee,5,NaN,"Muito bom, produto conforme anunciado, entrega antes do prazo.",2018-03-03 00:00:00,2018-03-04 22:56:47
99178,44d1e9165ec54b1d89d33594856af859,a7dbcf5043158d6fa72859eead2f3d10,4,NaN,NaN,2017-05-24 00:00:00,2017-05-24 23:15:21


In [166]:
cr = review[review.duplicated(subset='order_id', keep=False)][['order_id','review_score','review_answer_timestamp']].sort_values(by='order_id')
cr

,order_id,review_score,review_answer_timestamp
25612,0035246a40f520710769010f752e7507,5,2017-08-30 01:59:12
22423,0035246a40f520710769010f752e7507,5,2017-08-29 21:45:57
22779,013056cfe49763c6f66bda03396c5ee3,5,2018-02-23 12:12:30
68633,013056cfe49763c6f66bda03396c5ee3,4,2018-03-05 17:02:00
854,0176a6846bcb3b0d3aa3116a9a768597,5,2018-01-02 10:54:06
...,...,...,...
27465,ff763b73e473d03c321bcd5a053316e8,3,2017-11-18 09:02:48
41355,ff850ba359507b996e8b2fbb26df8d03,5,2017-08-17 11:56:55
18783,ff850ba359507b996e8b2fbb26df8d03,5,2017-08-25 11:40:22
92230,ffaabba06c9d293a3c614e0515ddbabc,3,2017-12-20 18:50:16


In [168]:
cr = review[review.duplicated(subset='review_id', keep=False)][['review_id','order_id','review_score','review_answer_timestamp']].sort_values(by='review_id')
cr

,review_id,order_id,review_score,review_answer_timestamp
46678,00130cbe1f9d422698c812ed8ded1919,dfcdfc43867d1c1381bfaf62d6b9c195,1,2018-03-20 18:08:23
29841,00130cbe1f9d422698c812ed8ded1919,04a28263e085d399c97ae49e0b477efa,1,2018-03-20 18:08:23
90677,0115633a9c298b6a98bcbe4eee75345f,78a4201f58af3463bdab842eea4bc801,5,2017-09-26 03:27:47
63193,0115633a9c298b6a98bcbe4eee75345f,0c9850b2c179c1ef60d2855e2751d1fa,5,2017-09-26 03:27:47
92876,0174caf0ee5964646040cd94e15ac95e,f93a732712407c02dce5dd5088d0f47b,1,2018-03-08 03:00:53
...,...,...,...,...
31120,fe5c833752953fed3209646f1f63b53c,4863e15fa53273cc7219c58f5ffda4fb,1,2018-02-28 13:57:52
7870,ff2fc9e68f8aabfbe18d710b83aabd30,2da58e0a7dcfa4ce1e00fad9d03ca3b5,2,2018-03-19 11:44:15
82521,ff2fc9e68f8aabfbe18d710b83aabd30,1078d496cc6ab9a8e6f2be77abf5091b,2,2018-03-19 11:44:15
73951,ffb8cff872a625632ac983eb1f88843c,c44883fc2529b4aa03ca90e7e09d95b6,3,2017-07-26 13:41:07


**olist_products_dataset_df**

**olist_sellers_dataset_df**

**product_category_name_translation_df**